## 1. Stock recommendation Model based on Technical indicators 
<p>This Module Fetch the data from https://in.tradingview.com/ . Main objective of this module is to take company symbol as input and provide recommendation for buying/Selling based on technical indicators data with different time intervals </p>
<p><em>Feel Free to reach out to me if you need any help in reading the data </em></p>

In [2]:
from tradingview_ta import TA_Handler, Interval, Exchange
import tradingview_ta
import pandas as pd
import numpy as np
from IPython.display import display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', None)


## 2. Build function for scrapping data from trading view handler
   <p> Updatehandler function will pull data from Exchange provided in input variables </p>

In [3]:
def updatehandler(interval,symbol,exchange,screener):    
    handler = TA_Handler(
        symbol=symbol,
        exchange=exchange,
        screener=screener,
        interval=interval,
        timeout=None )
    return handler


def get_analysis(symbol, interval,exchange,screener):
    handler = updatehandler(interval,symbol,exchange,screener)
    analysis = handler.get_analysis()
    return analysis


## 3. Build functions for recomandation after getting technical data from TA handler

In [6]:

def Trade_start(symbol_list,interval_list,exchange,screener):
    ##set index to 0 to travel through all symbols
    index = 0 
    row_counter = 0
    while index < len(symbol_list):
        symbol= symbol_list[index]
        #print('Stock Analysis started for '+symbol)
        for indx2 in range(len(interval_list)):
            interval= interval_list[indx2]
            #print('Stock Analysis interval used :'+interval)
            col_name = symbol+'_'+interval
            ##get analysis from trading view
            analysis = get_analysis(symbol, interval,exchange,screener)
            ##Update Stock_recommendation_details in pandas 
            if row_counter == 0:
                #print('Creating schema for new dataframe')
                columns_list = "Stock_Name,Close_Price,rsi_value,Interval,Refresh_Time,Overall_recommendation,oscillators_recommendation,moving_averages_recommendation".split(',')
                Stock_recommend_details = pd.DataFrame(columns = columns_list)
                ##create dataframe to store stock indicators values details 
                stock_indicator_details = pd.DataFrame.from_dict(analysis.indicators, orient ='index',columns = [col_name])
                stock_indicator_details[col_name] = np.round(stock_indicator_details[col_name], decimals = 3)
            ##get refreshed time from trading view    
            Refresh_Time =analysis.time.strftime("%m/%d/%Y, %H:%M:%S")
            ##update dataframe for recomandation
            Stock_recommend_details.loc[row_counter] = [symbol, analysis.indicators["close"], analysis.indicators["RSI"],interval, Refresh_Time,analysis.summary,analysis.oscillators,analysis.moving_averages]
            
            ##update stock indicators values details in pandas
            if row_counter > 0:
                #print('updating : ',col_name)
                stock_indicator_details[col_name] = pd.Series(analysis.indicators)
                stock_indicator_details = stock_indicator_details.replace(np.nan, 99.99)
                stock_indicator_details[col_name] = np.round(stock_indicator_details[col_name], decimals = 3)
            row_counter+=1
            
        index+=1
    return  stock_indicator_details,Stock_recommend_details


## Run Stock Analysis 
<P> This Engine can be executed for Stock/Crypto recommendation in following ways. <p>
<p> below are input parameters required for running the engine <p>
<p> trigger_mode -> it takes any value out of these two - stock / crypto <p>
<p> symbols ->  provide multiple or single stock/crypto symbol names which is listed on exchange, you can check symbols in trading view/any other trading platform Example -  ICICIBANK,INFY or PUSHUSDT  <p>
<p> exchange -> provide exchange names, it takes NSE as default value in case of stock and binance for cypto <p>
<p> screener -> provide screener value based on perticular country stock information you need EX - INDIA or USA or CRYPTO it takes default value as INDIA in case of stocks and CRYPTO in case of crypto <P> 
<p>interval_list -> it takes input as a list of interval data you need to look for recommendation. it takes default value as 
                     [ "1h","1d" ,"1W" ,"1M"] <p>

In [80]:

def main(symbols, trigger_mode ,exchange = "" ,screener  = "", interval = ""): 
    symbol_list = symbols.split(',')
    interval_list = interval.split(',')
    if interval == "" :
        interval_list = ["1h","1d" ,"1W" ,"1M"]
    if trigger_mode != 'crypto' :
        if exchange == ""  :
             exchange = "NSE"
        if screener == "" :
            screener="INDIA"
        stock_indicator_details,Stock_recommend_details = Trade_start(symbol_list,interval_list,exchange.upper(),screener)
        
    else :
        if exchange == ""  :
            exchange = "BINANCE"
        if screener == "" :
            screener="crypto"
        stock_indicator_details,Stock_recommend_details = Trade_start(symbol_list,interval_list,exchange.upper(),screener)

    return stock_indicator_details,Stock_recommend_details
    
#stock_indicator_details,Stock_recommend_details = main()    


## Reading output data 
<p> We get two following pandas dataframes as output from recommendation engine <p>
<p> stock_indicator_details -> this dataframe provides actual values of all the technical indicators <P>
<p> Stock_recommend_details -> this dataframe provide overall summary of buying/selling indication based on indicators data <p>

## Executing Stock recommendation engine
<p> Run Below cell to execute recommendation engine and provide required inputs <p>

In [69]:
trigger_mode = input("Provide trigger mode for engine, Pick b/w these two -> stock/crypto ->  : " ).lower()
symbols = input("Provide Symbols information, Ex-> ICICIBANK,INFY or PUSHUSDT : " ).upper()
exchange = input("Provide exchange information, keep it blank if you want use NSE as default for stock and Binance for crypto : " ) 
screener = input("Provide screener information, keep it blank if you want use INDIA as default for stock and crypto for crypto : " ) 
interval = input('''provide intervals with information, keep it blank if you want to use default value as  1h,1d,1W,1M for intervals ''')
stock_indicator_details,Stock_recommend_details = main(symbols, trigger_mode ,exchange,screener, interval ) 
display(Stock_recommend_details)

Provide trigger mode for engine, Pick b/w these two -> stock/crypto ->  : stock
Provide Symbols information, Ex-> ICICIBANK,INFY or PUSHUSDT : INFY
Provide exchange information, keep it blank if you want use NSE as default for stock and Binance for crypto : 
Provide screener information, keep it blank if you want use INDIA as default for stock and crypto for crypto : 
provide intervals with information, keep it blank if you want to use default value as  1h,1d,1W,1M for intervals 1h,1d
interval_list :  ['1h', '1d']


,Stock_Name,Close_Price,rsi_value,Interval,Refresh_Time,Overall_recommendation,oscillators_recommendation,moving_averages_recommendation
0,INFY,1724,27.835072,1h,"01/25/2022, 23:24:24","{'RECOMMENDATION': 'SELL', 'BUY': 5, 'SELL': 12, 'NEUTRAL': 9}","{'RECOMMENDATION': 'BUY', 'BUY': 3, 'SELL': 0, 'NEUTRAL': 8, 'COMPUTE': {'RSI': 'BUY', 'STOCH.K': 'NEUTRAL', 'CCI': 'NEUTRAL', 'ADX': 'NEUTRAL', 'AO': 'NEUTRAL', 'Mom': 'BUY', 'MACD': 'BUY', 'Stoch.RSI': 'NEUTRAL', 'W%R': 'NEUTRAL', 'BBP': 'NEUTRAL', 'UO': 'NEUTRAL'}}","{'RECOMMENDATION': 'STRONG_SELL', 'BUY': 2, 'SELL': 12, 'NEUTRAL': 1, 'COMPUTE': {'EMA10': 'SELL', 'SMA10': 'BUY', 'EMA20': 'SELL', 'SMA20': 'SELL', 'EMA30': 'SELL', 'SMA30': 'SELL', 'EMA50': 'SELL', 'SMA50': 'SELL', 'EMA100': 'SELL', 'SMA100': 'SELL', 'EMA200': 'SELL', 'SMA200': 'SELL', 'Ichimoku': 'NEUTRAL', 'VWMA': 'SELL', 'HullMA': 'BUY'}}"
1,INFY,1722.15,33.115611,1d,"01/25/2022, 23:24:25","{'RECOMMENDATION': 'SELL', 'BUY': 5, 'SELL': 14, 'NEUTRAL': 7}","{'RECOMMENDATION': 'NEUTRAL', 'BUY': 2, 'SELL': 3, 'NEUTRAL': 6, 'COMPUTE': {'RSI': 'NEUTRAL', 'STOCH.K': 'NEUTRAL', 'CCI': 'BUY', 'ADX': 'NEUTRAL', 'AO': 'SELL', 'Mom': 'SELL', 'MACD': 'SELL', 'Stoch.RSI': 'NEUTRAL', 'W%R': 'BUY', 'BBP': 'NEUTRAL', 'UO': 'NEUTRAL'}}","{'RECOMMENDATION': 'STRONG_SELL', 'BUY': 3, 'SELL': 11, 'NEUTRAL': 1, 'COMPUTE': {'EMA10': 'SELL', 'SMA10': 'SELL', 'EMA20': 'SELL', 'SMA20': 'SELL', 'EMA30': 'SELL', 'SMA30': 'SELL', 'EMA50': 'SELL', 'SMA50': 'SELL', 'EMA100': 'SELL', 'SMA100': 'SELL', 'EMA200': 'BUY', 'SMA200': 'BUY', 'Ichimoku': 'NEUTRAL', 'VWMA': 'SELL', 'HullMA': 'BUY'}}"


### Run ICICIBANK recommendation based on 1-day chart

In [76]:
trigger_mode = 'stock'
symbols = "ICICIBANK"
interval = '1d'
exchange = "NSE"
screener = "INDIA"
stock_indicator_details,Stock_recommend_details = main(symbols, trigger_mode ,exchange  ,screener   , interval ) 
display(Stock_recommend_details)

interval_list :  ['1d']


,Stock_Name,Close_Price,rsi_value,Interval,Refresh_Time,Overall_recommendation,oscillators_recommendation,moving_averages_recommendation
0,ICICIBANK,801.65,58.30968,1d,"01/25/2022, 23:34:24","{'RECOMMENDATION': 'BUY', 'BUY': 14, 'SELL': 3, 'NEUTRAL': 9}","{'RECOMMENDATION': 'NEUTRAL', 'BUY': 2, 'SELL': 1, 'NEUTRAL': 8, 'COMPUTE': {'RSI': 'NEUTRAL', 'STOCH.K': 'NEUTRAL', 'CCI': 'NEUTRAL', 'ADX': 'NEUTRAL', 'AO': 'NEUTRAL', 'Mom': 'BUY', 'MACD': 'SELL', 'Stoch.RSI': 'NEUTRAL', 'W%R': 'NEUTRAL', 'BBP': 'BUY', 'UO': 'NEUTRAL'}}","{'RECOMMENDATION': 'STRONG_BUY', 'BUY': 12, 'SELL': 2, 'NEUTRAL': 1, 'COMPUTE': {'EMA10': 'SELL', 'SMA10': 'SELL', 'EMA20': 'BUY', 'SMA20': 'BUY', 'EMA30': 'BUY', 'SMA30': 'BUY', 'EMA50': 'BUY', 'SMA50': 'BUY', 'EMA100': 'BUY', 'SMA100': 'BUY', 'EMA200': 'BUY', 'SMA200': 'BUY', 'Ichimoku': 'NEUTRAL', 'VWMA': 'BUY', 'HullMA': 'BUY'}}"


###  Run Crypto Analysis for BTCUSDT on 30min and 1 hr chart data

In [83]:
trigger_mode = 'crypto'
symbols = "BTCUSDT"
interval = '30m,1h'
exchange = "binance"
screener = "crypto"
stock_indicator_details,Stock_recommend_details = main(symbols, trigger_mode ,exchange  ,screener   , interval ) 
display(Stock_recommend_details)


,Stock_Name,Close_Price,rsi_value,Interval,Refresh_Time,Overall_recommendation,oscillators_recommendation,moving_averages_recommendation
0,BTCUSDT,37184.01,64.476852,30m,"01/25/2022, 23:40:38","{'RECOMMENDATION': 'BUY', 'BUY': 15, 'SELL': 4, 'NEUTRAL': 7}","{'RECOMMENDATION': 'NEUTRAL', 'BUY': 2, 'SELL': 3, 'NEUTRAL': 6, 'COMPUTE': {'RSI': 'NEUTRAL', 'STOCH.K': 'NEUTRAL', 'CCI': 'SELL', 'ADX': 'NEUTRAL', 'AO': 'NEUTRAL', 'Mom': 'BUY', 'MACD': 'BUY', 'Stoch.RSI': 'SELL', 'W%R': 'SELL', 'BBP': 'NEUTRAL', 'UO': 'NEUTRAL'}}","{'RECOMMENDATION': 'STRONG_BUY', 'BUY': 13, 'SELL': 1, 'NEUTRAL': 1, 'COMPUTE': {'EMA10': 'BUY', 'SMA10': 'BUY', 'EMA20': 'BUY', 'SMA20': 'BUY', 'EMA30': 'BUY', 'SMA30': 'BUY', 'EMA50': 'BUY', 'SMA50': 'BUY', 'EMA100': 'BUY', 'SMA100': 'BUY', 'EMA200': 'BUY', 'SMA200': 'BUY', 'Ichimoku': 'NEUTRAL', 'VWMA': 'BUY', 'HullMA': 'SELL'}}"
1,BTCUSDT,37184.01,64.434024,1h,"01/25/2022, 23:40:39","{'RECOMMENDATION': 'BUY', 'BUY': 13, 'SELL': 5, 'NEUTRAL': 8}","{'RECOMMENDATION': 'SELL', 'BUY': 1, 'SELL': 3, 'NEUTRAL': 7, 'COMPUTE': {'RSI': 'NEUTRAL', 'STOCH.K': 'NEUTRAL', 'CCI': 'SELL', 'ADX': 'NEUTRAL', 'AO': 'NEUTRAL', 'Mom': 'SELL', 'MACD': 'BUY', 'Stoch.RSI': 'NEUTRAL', 'W%R': 'SELL', 'BBP': 'NEUTRAL', 'UO': 'NEUTRAL'}}","{'RECOMMENDATION': 'STRONG_BUY', 'BUY': 12, 'SELL': 2, 'NEUTRAL': 1, 'COMPUTE': {'EMA10': 'BUY', 'SMA10': 'BUY', 'EMA20': 'BUY', 'SMA20': 'BUY', 'EMA30': 'BUY', 'SMA30': 'BUY', 'EMA50': 'BUY', 'SMA50': 'BUY', 'EMA100': 'BUY', 'SMA100': 'BUY', 'EMA200': 'SELL', 'SMA200': 'SELL', 'Ichimoku': 'NEUTRAL', 'VWMA': 'BUY', 'HullMA': 'BUY'}}"
